In [1]:
import random
import torch

%config Completer.use_jedi = False

In [3]:
test_string = "This is a test string. It is a test string. A very nice test string. I hope you enjoy this test string." #We want to pretrain an AI system on text data
block_size = 128 #This is the size of the tensor we send the sentence into

In [64]:
'''
# I'm writing an AI model that takes in wikipedia data on famous people and tries to predict
# from a description where the famous person was born
# The data looks like this:

x: Where was Khatchig Mouradian born?⁇Lebanon⁇□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□

# The answer is at the end of the sentence, and then we take the sentence out, and the answer just becomes the target

y: □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□⁇Lebanon⁇□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□

# We want to "corrupt" the sentence, so the first thing we do is randomly truncate the sentence
# but I want to make sure it's not smaller than 4 characters, and not longer than 7/8 of the block_size
'''

document = test_string[:random.randint(4,int(block_size*7/8))]
print(test_string)
print(document)

This is a test string. It is a test string. A very nice test string. I hope you enjoy this test string.
This is a test string. It is a test string. A


In [65]:
MASK_CHAR = u"\u2047" #Double Question Mark Character
PAD_CHAR = u"\u25A1" #Pad character

# I'm just generating 10 random corruptions, which is why I'm using a for loop
for i in range(10):
    #Re-truncating so you can see the effect on each sentence
    document = test_string[:random.randint(4,int(block_size*7/8))]
    #Sample a random window size, which will become the size of the target
    r_size = random.gauss(.25, .03)
    #Find the center index of the target
    r_idx = random.uniform((r_size/2),1-(r_size/2))
    #place the first mask character r_size/2 below the center index
    mask_1 = int(len(document)*(r_idx-(r_size/2)))
    #Plac e the seond mask character r_size/2 below the center index
    mask_2 = int(len(document)*(r_idx+(r_size/2)))
    #Join the parts together and pad the remaining white space (block_size - length of document -2)
    #Subtract 2 because we added 2 masked characters
    print(''.join([document[:mask_1], MASK_CHAR, document[mask_1:mask_2],
        MASK_CHAR, document[mask_2:], PAD_CHAR*(block_size-len(document)-2)]))

This is a test string. It is a test string. A very nice test string. ⁇I hope you enjoy this test st⁇ring.□□□□□□□□□□□□□□□□□□□□□□□
T⁇his is a ⁇test string. It is a test s□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
This is a test string. It is a test string. A ⁇very nice test s⁇tring□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
Thi⁇s ⁇is a t□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
This is a test string. ⁇It is a test str⁇ing. A very nice test string. □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
This is a test string. It is a test strin⁇g. A very nice test s⁇tring. I hope you enjoy this test string.□□□□□□□□□□□□□□□□□□□□□□□
This is a test string.⁇ It is a t⁇est strin□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□
This is a test string. It is a test s⁇tring. A very ⁇nice test □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□

In [261]:
X = torch.randn(128, 128, 256)

# B = Batch Size
# T = Sequence Length
# C = Embeding Size
B, T, C = X.size()

n_embd = 256
n_head = 8
block_size = 128

In [262]:
w1 = torch.nn.Linear(n_embd, n_embd)
w2 = torch.nn.Parameter(torch.zeros(n_embd // n_head, block_size-1))
b2 = torch.nn.Parameter(torch.zeros(block_size-1))
value = torch.nn.Linear(n_embd, n_embd)
torch.nn.init.uniform_(w2,-0.001,0.001)

Parameter containing:
tensor([[-7.2175e-04, -4.1507e-04, -6.6582e-04,  ..., -5.3857e-04,
          5.7601e-04,  2.3573e-04],
        [-4.1511e-04, -3.0242e-04, -7.4655e-04,  ...,  5.7731e-04,
         -7.8054e-04, -3.0395e-04],
        [-8.4347e-04, -3.5446e-04, -4.9797e-04,  ...,  5.9881e-04,
          9.7359e-04,  4.0340e-04],
        ...,
        [-6.3570e-04, -6.0424e-04,  4.3990e-04,  ..., -1.0673e-04,
          1.1454e-05,  7.5907e-04],
        [-7.8225e-04,  5.6429e-04,  3.5070e-04,  ...,  5.6131e-04,
         -5.3707e-04, -4.8799e-04],
        [ 3.8860e-04, -5.1318e-04,  5.4712e-04,  ..., -1.0840e-04,
          8.9250e-04, -7.9351e-04]], requires_grad=True)

In [275]:
# We want a TxT output

wA = torch.relu(w1(X).view(B, T, n_head, C // n_head).transpose(1,2))
print(wA.shape)
print(w2.unsqueeze(0).unsqueeze(0).shape)
print((wA @ w2.unsqueeze(0).unsqueeze(0) + b2).shape)
v = value(X).view(B, T, n_head, C // n_head).transpose(1, 2)
print(v.shape)
print(v.transpose(-2, -1).shape)

torch.Size([128, 8, 128, 32])
torch.Size([1, 1, 32, 127])
torch.Size([128, 8, 128, 127])
torch.Size([128, 8, 128, 32])
torch.Size([128, 8, 32, 128])


In [274]:
y = att @ v
print(y.transpose(1, 2).shape)
y = y.transpose(1, 2).contiguous().view(B, T, C)

RuntimeError: Expected tensor to have size 127 at dimension 1, but got size 128 for argument #2 'batch2' (while checking arguments for bmm)

In [246]:
y.shape

torch.Size([128, 8, 128, 128])